### sumatorio del numero de enfermedades

In [ ]:
db["clinicos.2"].aggregate([
  {
    $addFields: {
      total_patologias: {
        $add: [
          "$diabetes", "$hipertension", "$obesidad", "$cancer",
          "$enfermedad_cardiaca", "$asma", "$epoc"
        ]
      }
    }
  },
  {
    $project: {
      _id: 0,
      paciente_id: 1,
      total_patologias: 1
    }
  }
])

### devuelve glucosa, codigo postal y mayor/joven en función de si es >/< 50 años

In [ ]:
db["bioquimicos.1"].aggregate([
  {
    $lookup: {
      from: "sociodemograficos.6",
      localField: "paciente_id",
      foreignField: "paciente_id",
      as: "socio"
    }
  },
  {
    $unwind: "$socio"
  },
  {
    $project: {
      _id: 0,
      paciente_id: 1,
      glucosa: 1,
      codigo_postal: "$socio.codigo_postal",
      grupo_edad: {
        $cond: {
          if: { $gt: ["$socio.edad", 50] },
          then: "mayor",
          else: "joven"
        }
      }
    }
  }
])

#### igual pero juntando 3 tablas

In [ ]:
db["bioquimicos.1"].aggregate([
  {
    $lookup: {
      from: "sociodemograficos.6",
      localField: "paciente_id",
      foreignField: "paciente_id",
      as: "socio"
    }
  },
  { $unwind: "$socio" },

  {
    $lookup: {
      from: "clinicos.2",
      localField: "paciente_id",
      foreignField: "paciente_id",
      as: "clinico"
    }
  },
  { $unwind: "$clinico" },

  {
    $project: {
      _id: 0,
      paciente_id: 1,
      glucosa: 1,
      codigo_postal: "$socio.codigo_postal",
      grupo_edad: {
        $cond: {
          if: { $gt: ["$socio.edad", 50] },
          then: "mayor",
          else: "joven"
        }
      },
      total_diagnosticos: {
        $add: [
          "$clinico.diabetes",
          "$clinico.hipertension",
          "$clinico.obesidad",
          "$clinico.cancer",
          "$clinico.enfermedad_cardiaca",
          "$clinico.asma",
          "$clinico.epoc"
        ]
      }
    }
  }
])